<a href="https://colab.research.google.com/github/EjbejaranosAI/EmotionUnify/blob/main/Meld_feature_extractionV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Download andinstall libraries and dataset

In [ ]:
!pip install transformers lazypredict

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 58.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 35.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 85.1 MB/s eta 0:00:00


### Module to set the dataset

In [ ]:
"""
This module is to download multimodal datasets and store them in the dataset directory
"""
import os
import logging
import shutil
import subprocess
import tarfile


def download_meld_dataset():
    """
    Download MELD dataset and store it into the dataset directory

    """
    os.chdir('/content/datasets')
    try:
        # Add logging configuration
        logging.basicConfig(filename='download.log', level=logging.INFO)
        logging.info('Downloading MELD dataset')

        # Use wget to download the dataset
        try:
            subprocess.run(['wget', 'https://huggingface.co/datasets/declare-lab/MELD/resolve/main/MELD.Raw.tar.gz'])
        except:
            subprocess.run(['wget','http://web.eecs.umich.edu/~mihalcea/downloads/MELD.Raw.tar.gz'])

        # Extract the downloaded dataset
        subprocess.run(['tar', '-xvf', 'MELD.Raw.tar.gz'])

        # Remove the downloaded tar.gz file
        os.remove('MELD.Raw.tar.gz')

        # Rename the extracted directory to "MELD"
        os.rename('MELD.Raw', 'MELD')

        # Extract the inner tar files
        files = os.listdir("./MELD")
        files_to_uncompress = []
        for file in files:
            if file.endswith('.tar.gz'):
                print("tar.gz file found")
                files_to_uncompress.append(file)
        if len(files_to_uncompress) >0:
            for file in files_to_uncompress:
                # I want to extract the files in the same directory that was found it and later deleter the tar.gz files
                tar = tarfile.open("./MELD/"+file)
                tar.extractall("./MELD/")
                tar.close()
                os.remove("./MELD/"+file)

        else:
            print("No tar.gz files found")
    except:
        logging.error('Error in downloading MELD dataset')
        print('Error in downloading MELD dataset')


if __name__ == "__main__":
    download_meld_dataset()

tar.gz file found
tar.gz file found
tar.gz file found


In [1]:
import os
import numpy as np
import pandas as pd

path_dataset = "/content/datasets/MELD"

# In[2]:
def load_data(path):
    csv_file_data = []
    files_to_uncompress = []
    files = os.listdir(path)
    print(files)
    # get the csv files from the files and read it
    for file in files:
        if file.endswith(".csv"):
            print(file)
            csv_file_data.append({file:pd.read_csv(path + '/' + file)})


    print(len(csv_file_data))



    return files, csv_file_data

files , csv_file_data = load_data(path_dataset)

['output_repeated_splits_test', 'README.txt', '._train_splits', 'train_sent_emo.csv', 'train_splits', 'dev_sent_emo.csv', '._output_repeated_splits_test', 'dev_splits_complete', 'test_sent_emo.csv']
train_sent_emo.csv
dev_sent_emo.csv
test_sent_emo.csv
3


In [2]:
train_df = csv_file_data[0]['train_sent_emo.csv']
dev_df = csv_file_data[1]['dev_sent_emo.csv']
test_df = csv_file_data[2]['test_sent_emo.csv']

In [3]:
df_train = train_df
df_dev = dev_df
df_test = test_df
print(len(df_train))
df_train.head()

9989


,Sr No.,Utterance,Speaker,Emotion,Sentiment,Dialogue_ID,Utterance_ID,Season,Episode,StartTime,EndTime
0,1,also I was the point person on my companys tr...,Chandler,neutral,neutral,0,0,8,21,"00:16:16,059","00:16:21,731"
1,2,You mustve had your hands full.,The Interviewer,neutral,neutral,0,1,8,21,"00:16:21,940","00:16:23,442"
2,3,That I did. That I did.,Chandler,neutral,neutral,0,2,8,21,"00:16:23,442","00:16:26,389"
3,4,So lets talk a little bit about your duties.,The Interviewer,neutral,neutral,0,3,8,21,"00:16:26,820","00:16:29,572"
4,5,My duties? All right.,Chandler,surprise,positive,0,4,8,21,"00:16:34,452","00:16:40,917"


In [4]:
# Define mapping dictionaries
emotion_mapping = {
    'neutral': [1., 0., 0., 0., 0., 0., 0.],
    'surprise': [0., 1., 0., 0., 0., 0., 0.],
    'fear': [0., 0., 1., 0., 0., 0., 0.],
    'sadness': [0., 0., 0., 1., 0., 0., 0.],
    'joy': [0., 0., 0., 0., 1., 0., 0.],
    'disgust': [0., 0., 0., 0., 0., 1., 0.],
    'anger': [0., 0., 0., 0., 0., 0., 1.]
}

sentiment_mapping = {
    'neutral': 0,
    'positive': 1,
    'negative': 2
}

# Feature extraction in text

In [7]:
from transformers import BertTokenizer, BertModel
import torch
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import numpy as np
from tqdm import tqdm  # Import tqdm for progress bars

# Define a function to extract BERT features
def extract_bert_features(df):
    # Tokenize utterances and obtain BERT features with padding
    max_length = 128  # Define your desired maximum sequence length

    # Tokenize utterances and obtain BERT features
    utterances = df['Utterance'].tolist()
    input_ids = []
    attention_masks = []

    # Use tqdm to add progress bars
    for utterance in tqdm(utterances, desc="Tokenizing Utterances"):
        tokens = tokenizer.encode(utterance, add_special_tokens=True, max_length=max_length, truncation=True, padding='max_length')
        input_ids.append(tokens)
        attention_masks.append([1] * len(tokens))

    input_ids = torch.tensor(input_ids)
    attention_masks = torch.tensor(attention_masks)

    with torch.no_grad():
        bert_outputs = model(input_ids, attention_mask=attention_masks)

    # Use BERT features to train a logistic regression classifier
    X = bert_outputs[0][:, 0, :].numpy()  # Using the [CLS] token's representation
    y = df['Sentiment_encoded'].tolist()

    return X, y

In [ ]:
# Load pre-trained BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Process the training, development, and test datasets
df_train['video_id'] = df_train.apply(lambda row: f'dia{row["Dialogue_ID"]}_utt{row["Utterance_ID"]}.mp4', axis=1)
df_dev['video_id'] = df_dev.apply(lambda row: f'dia{row["Dialogue_ID"]}_utt{row["Utterance_ID"]}.mp4', axis=1)
df_test['video_id'] = df_test.apply(lambda row: f'dia{row["Dialogue_ID"]}_utt{row["Utterance_ID"]}.mp4', axis=1)

df_train['Emotion_encoded'] = df_train['Emotion'].map(emotion_mapping)
df_train['Sentiment_encoded'] = df_train['Sentiment'].map(sentiment_mapping)
df_processed_train = df_train[['video_id', 'Utterance', 'Emotion_encoded', 'Sentiment_encoded']]

df_dev['Emotion_encoded'] = df_dev['Emotion'].map(emotion_mapping)
df_dev['Sentiment_encoded'] = df_dev['Sentiment'].map(sentiment_mapping)
df_processed_dev = df_dev[['video_id', 'Utterance', 'Emotion_encoded', 'Sentiment_encoded']]

df_test['Emotion_encoded'] = df_test['Emotion'].map(emotion_mapping)
df_test['Sentiment_encoded'] = df_test['Sentiment'].map(sentiment_mapping)
df_processed_test = df_test[['video_id', 'Utterance', 'Emotion_encoded', 'Sentiment_encoded']]

# Extract BERT features for the training, development, and test datasets
X_train, y_train = extract_bert_features(df_processed_train)
np.save('train_text_features.npy', X_train)

X_dev, y_dev = extract_bert_features(df_processed_dev)
np.save('dev_text_features.npy', X_dev)

X_test, y_test = extract_bert_features(df_processed_test)
np.save('test_text_features.npy', X_test)

Tokenizing Utterances: 100%|██████████| 9989/9989 [00:02<00:00, 3952.59it/s]


In [ ]:


# Load pre-trained BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Tokenize utterances and obtain BERT features with padding
max_length = 128  # Define your desired maximum sequence length

# Tokenize utterances and obtain BERT features
utterances = df_processed_train['Utterance'].tolist()
input_ids = []
attention_masks = []

# Use tqdm to add progress bars
for utterance in tqdm(utterances, desc="Tokenizing Utterances"):
    tokens = tokenizer.encode(utterance, add_special_tokens=True, max_length=max_length, truncation=True, padding='max_length')
    input_ids.append(tokens)
    attention_masks.append([1] * len(tokens))

input_ids = torch.tensor(input_ids)
attention_masks = torch.tensor(attention_masks)

with torch.no_grad():
    bert_outputs = model(input_ids, attention_mask=attention_masks)

# Use BERT features to train a logistic regression classifier
X = bert_outputs[0][:, 0, :].numpy()  # Using the [CLS] token's representation
y = df_processed_train['Sentiment_encoded'].tolist()

Tokenizing Utterances: 100%|██████████| 9989/9989 [00:02<00:00, 3894.53it/s]


### Text classifier

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert one-hot encoded labels to single-class labels
#y_train_single = np.argmax(y_train, axis=1)
#y_test_single = np.argmax(y_test, axis=1)


classifier = LogisticRegression(max_iter=1000)
classifier.fit(X_train, y_train)

# Evaluate the classifier
y_pred = classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

Accuracy: 0.6766766766766766


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
from lazypredict.Supervised import LazyClassifier

clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(X_train, X_test, y_train, y_test)

print(models)

 97%|█████████▋| 28/29 [09:13<00:25, 25.87s/it]

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.022250 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 195839
[LightGBM] [Info] Number of data points in the train set: 7991, number of used features: 768
[LightGBM] [Info] Start training from score -0.763249
[LightGBM] [Info] Start training from score -1.444388
[LightGBM] [Info] Start training from score -1.210795


100%|██████████| 29/29 [09:23<00:00, 19.44s/it]

                               Accuracy  Balanced Accuracy ROC AUC  F1 Score  \
Model                                                                          
LinearDiscriminantAnalysis         0.67               0.62    None      0.66   
LogisticRegression                 0.67               0.62    None      0.66   
RidgeClassifierCV                  0.67               0.61    None      0.66   
RidgeClassifier                    0.67               0.61    None      0.66   
LinearSVC                          0.64               0.59    None      0.64   
SGDClassifier                      0.63               0.58    None      0.62   
CalibratedClassifierCV             0.66               0.58    None      0.63   
SVC                                0.65               0.57    None      0.63   
XGBClassifier                      0.64               0.57    None      0.62   
LGBMClassifier                     0.64               0.56    None      0.62   
PassiveAggressiveClassifier        0.60 

In [ ]:
models

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
LinearDiscriminantAnalysis,0.67,0.62,None,0.66,1.51
LogisticRegression,0.67,0.62,None,0.66,1.08
RidgeClassifierCV,0.67,0.61,None,0.66,1.49
RidgeClassifier,0.67,0.61,None,0.66,0.36
LinearSVC,0.64,0.59,None,0.64,52.50
SGDClassifier,0.63,0.58,None,0.62,5.46
CalibratedClassifierCV,0.66,0.58,None,0.63,202.60
SVC,0.65,0.57,None,0.63,28.18
XGBClassifier,0.64,0.57,None,0.62,83.09


# Feature extraction in audio

In [ ]:
import torch
import torch.nn as nn
from transformers import Wav2Vec2Processor
from transformers.models.wav2vec2.modeling_wav2vec2 import (
    Wav2Vec2Model,
    Wav2Vec2PreTrainedModel,
)
import torchaudio


class RegressionHead(nn.Module):
    r"""Classification head."""

    def __init__(self, config):

        super().__init__()

        self.dense = nn.Linear(config.hidden_size, config.hidden_size)
        self.dropout = nn.Dropout(config.final_dropout)
        self.out_proj = nn.Linear(config.hidden_size, config.num_labels)

    def forward(self, features, **kwargs):

        x = features
        x = self.dropout(x)
        x = self.dense(x)
        x = torch.tanh(x)
        x = self.dropout(x)
        x = self.out_proj(x)

        return x


class EmotionModel(Wav2Vec2PreTrainedModel):
    r"""Speech emotion classifier."""

    def __init__(self, config):

        super().__init__(config)

        self.config = config
        self.wav2vec2 = Wav2Vec2Model(config)
        self.classifier = RegressionHead(config)
        self.init_weights()

    def forward(
            self,
            input_values,
    ):

        outputs = self.wav2vec2(input_values)
        hidden_states = outputs[0]
        hidden_states = torch.mean(hidden_states, dim=1)
        logits = self.classifier(hidden_states)

        return hidden_states, logits


def process_func(
    wavs,
    sampling_rate: int
    # embeddings: bool = False,
):
    r"""Predict emotions or extract embeddings from raw audio signal."""

    # run through processor to normalize signal
    # always returns a batch, so we just get the first entry
    # then we put it on the device
    # wavs = pad_sequence(wavs, batch_first=True)
    # load model from hub
    device = 'cpu'
    model_name = 'audeering/wav2vec2-large-robust-12-ft-emotion-msp-dim'
    processor = Wav2Vec2Processor.from_pretrained(model_name)
    model = EmotionModel.from_pretrained(model_name)

    y = processor([wav.cpu().numpy() for wav in wavs],
                   sampling_rate=sampling_rate,
                   return_tensors="pt",
                   padding="longest"
        )
    y = y['input_values']
    y = y.to(device)


    y = model(y)

    return {
        'hidden_states': y[0],
        'logits': y[1],
    }




### Extracting features for one audio

In [77]:
import moviepy.editor as mpe
import torchaudio

# Load the video file
video_file = "/content/datasets/MELD/train_splits/dia125_utt3.mp4"
video_clip = mpe.VideoFileClip(video_file)

# Extract the audio
audio_clip = video_clip.audio

# Save the audio as a WAV file
audio_file = "/content/audio.wav"
audio_clip.write_audiofile(audio_file, codec='pcm_s16le')

## test to an audiofile
sampling_rate = 16000
signal = [torchaudio.load('/content/tmp/audio.wav')[0].squeeze().to('cpu') for _ in range(2)]
# Reshape the input tensors to have shape [sequence_length].
signal = [tensor.view(-1) for tensor in signal]
# Extract hidden states.
with torch.no_grad():
    hs = process_func(signal, sampling_rate)['hidden_states']

print(f"Hidden states shape={hs.shape}")

OSError: ignored

### Extracting features in all the dataset

In [ ]:
def extract_audio_features_from_video(video_path):
  audio_features = []
  video_clip = mpe.VideoFileClip(video_path)
  # Extract the audio
  audio_clip = video_clip.audio
  # Save the audio as a WAV file
  audio_file = "/content/audio.wav"
  audio_clip.write_audiofile(audio_file, codec='pcm_s16le')
  ## test to an audiofile
  sampling_rate = 16000
  signal = [torchaudio.load('/content/audio.wav')[0].squeeze().to('cpu') for _ in range(2)]
  # Reshape the input tensors to have shape [sequence_length].
  signal = [tensor.view(-1) for tensor in signal]
  # Extract hidden states.
  with torch.no_grad():
    audio_features = process_func(signal, sampling_rate)

  return audio_features['hidden_states']

video_folder = "/content/datasets/MELD/train_splits"
video_id_files = df_processed['video_id'].tolist()


audio_features = []
for video_id in tqdm(video_id_files):
    video_path = os.path.join(video_folder, video_id)
    audio_features.append(extract_audio_features_from_video(video_path))

# Save the extracted audio features to a file.
np.save('audio_features.npy', audio_features)


  0%|          | 0/9989 [00:00<?, ?it/s]

MoviePy - Writing audio in /content/audio.wav



  0%|          | 0/9989 [00:00<?, ?it/s]

MoviePy - Done.


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
  0%|          | 1/9989 [00:08<24:22:38,  8.79s/it]

...

MoviePy - Writing audio in /content/audio.wav



 12%|█▏        | 1164/9989 [2:32:01<23:48:39,  9.71s/it]

MoviePy - Done.


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
 12%|█▏        | 1165/9989 [2:32:06<19:12:04,  7.83s/it]


OSError: ignored

In [ ]:
df_processed[]

In [73]:
!ls /content/datasets/MELD/train_splits/dia125_utt3.mp4

MELD
